In [1]:
#Intro to Torch work

In [2]:
import torch
import numpy as np

In [5]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)
x_data

tensor([[1, 2],
        [3, 4]])

In [7]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)

In [14]:
shape = (2,3)
torch.rand(shape)
tensor = torch.ones(2,3)
tensor.device

device(type='cpu')

In [15]:
torch.cuda.is_available()

True

In [16]:
if torch.cuda.is_available():
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

Device tensor is stored on: cuda:0


In [31]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

In [33]:
prediction = model(data)
loss = (prediction - labels).sum()
loss.backward() 
prediction

tensor([[-7.6936e-01, -4.7470e-01, -7.7365e-01, -1.8096e+00, -9.2946e-01,
         -2.1604e-01, -5.3495e-01,  6.4181e-01,  6.3297e-01, -6.4690e-01,
         -5.7755e-01, -5.1415e-01, -1.0975e-02, -7.6475e-01, -1.0861e+00,
         -5.7404e-01, -7.0902e-01,  2.6921e-02, -3.7373e-01, -1.4492e-01,
         -1.5400e+00, -7.7540e-01, -1.4056e+00, -7.2108e-02, -9.6053e-01,
         -1.2402e+00, -1.0737e+00, -1.3540e+00, -9.8589e-01, -2.5344e-01,
         -1.0027e+00, -6.6669e-01, -5.7755e-01, -4.7633e-01, -4.6286e-01,
         -4.7300e-01,  5.6732e-01, -7.1667e-01, -3.2017e-01,  1.4508e-01,
         -9.1095e-01, -9.6330e-01, -9.3278e-01, -2.0563e-01, -8.4864e-01,
         -4.0223e-01, -8.3286e-01, -5.0122e-01, -1.3263e+00, -1.1246e+00,
         -6.8112e-01,  2.8575e-01, -5.1178e-01, -8.3557e-01, -1.9911e-01,
         -1.2955e+00, -1.5986e-01, -1.7336e+00, -7.4570e-01, -4.9299e-01,
          8.0457e-01,  1.2843e-01, -2.0775e-01,  1.9774e-01, -9.2462e-01,
         -2.5730e-01, -3.0241e-01, -2.

In [34]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [35]:
optim.step() #gradient descent

In [36]:
model(data)

tensor([[ -9.9961,  -9.7858,  -9.8884, -11.0517, -10.3028,  -9.8727,  -9.9665,
          -8.6391,  -9.0389, -10.1599, -10.2207,  -9.9898,  -9.4981, -10.1876,
         -10.3829, -10.0036, -10.4114,  -9.3333,  -9.6983,  -9.4883, -10.8499,
         -10.2680, -10.8709,  -9.4191, -10.1939, -10.3498, -10.0295, -10.3315,
         -10.0532,  -9.6892, -10.1193, -10.0840,  -9.8188,  -9.8327,  -9.7211,
          -9.7313,  -8.6692,  -9.8922,  -9.5384,  -8.9775, -10.0276, -10.1461,
         -10.1908,  -9.5130,  -9.9184,  -9.6942,  -9.9121,  -9.7771, -10.5993,
         -10.3735,  -9.9601,  -9.1004,  -9.6527,  -9.7040,  -9.5601, -10.4115,
          -9.5276, -10.6021,  -9.7510,  -9.8299,  -8.4517,  -9.2089,  -9.5766,
          -9.2477, -10.2333,  -9.5723,  -9.7039,  -9.7148, -10.2486, -10.4907,
         -11.0234,  -8.6975, -10.6354,  -9.4622, -10.2414, -10.4325,  -9.0119,
          -9.8618,  -8.8631,  -9.0092, -10.3223, -10.9597,  -9.5410, -10.1833,
          -9.8941,  -9.5399,  -9.3954,  -8.9407,  -9

In [1]:
import numpy as np
import sigkernel
import torch
from einops import rearrange
from itertools import product
import cython

In [5]:

y_obs = np.array([[[2],[2],[3]]]) #Needa 1 Dim, 3 timesteps, 1 output per timestep
x_for = np.array([[[2],[100],[3]],[[2],[2],[3]]]) #2 Models, each with 3 timesteps, and 1 output per timestep

#if one of the models is the same as the path, value doesn't adjust

y_obs = np.cumsum(y_obs, axis=1)
x_for = np.cumsum(x_for, axis=1)

X = torch.tensor(x_for, dtype=torch.double)
y = torch.tensor(y_obs, dtype=torch.double)

#y_obs = np.cumsum(y_obs)
#y_for = np.cumsum(y_for)

static_kernel = sigkernel.RBFKernel(sigma=2)
dyadic_order = 4
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

K_XX = signature_kernel.compute_Gram(X,X)


#sr = signature_kernel.compute_scoring_rule(X,y)
#sr

# """Input:
#             - X: torch tensor of shape (batch, length_X, dim),
#             - y: torch tensor of shape (1, length_Y, dim)
#     Output:
#             - signature kernel scoring rule S(X,y) = E[k(X,X)] - 2E[k(X,y]
# """

# K_XX = self.compute_Gram(X, X, sym=True, max_batch=max_batch)
# K_Xy = self.compute_Gram(X, y, sym=False, max_batch=max_batch)

# K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))

# return K_XX_m - 2. * torch.mean(K_Xy)


In [119]:
y_obs = np.array([[[2],[1],[5]]]) #Needa 1 Dim, 3 timesteps, 1 output per timestep
x_for = np.array([[[2],[1],[5]],[[2],[99],[500]],[[2],[1],[5]], [[2],[1],[5]]]) #2 Models, each with 3 timesteps, and 1 output per timestep

#if one of the models is the same as the path, value doesn't adjust

y_obs = np.cumsum(y_obs, axis=1)
x_for = np.cumsum(x_for, axis=1)

X = torch.tensor(x_for, dtype=torch.double)
y = torch.tensor(y_obs, dtype=torch.double)

#print("X:", X)
#print("y:", y)

static_kernel = sigkernel.RBFKernel(sigma=3)
dyadic_order = 2
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

K_XX = signature_kernel.compute_Gram(X,X)
K_Xy = signature_kernel.compute_Gram(X, y)
K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))
print(K_XX)
print(K_Xy) #
print("K_XX_m:", K_XX_m) #Expected self similirity of predictions... 
print(-2. * torch.mean(K_Xy)) #Expected similarity between prediction and observation
print(K_XX_m -(2.*torch.mean(K_Xy)))
#Rewards internal consistency + accuracy
#Relatively low S(X,y) indicate predicted paths are consistent

#Sometimes multiple paths lead to same score when increase to internal similarity first term increases, but if it make it more accurate second term increases as well. 

tensor([[5.3451, 2.2806, 5.3451, 5.3451],
        [2.2806, 8.2111, 2.2806, 2.2806],
        [5.3451, 2.2806, 5.3451, 5.3451],
        [5.3451, 2.2806, 5.3451, 5.3451]], dtype=torch.float64)
tensor([[5.3451],
        [2.2806],
        [5.3451],
        [5.3451]], dtype=torch.float64)
K_XX_m: tensor(3.8129, dtype=torch.float64)
tensor(-9.1580, dtype=torch.float64)
tensor(-5.3451, dtype=torch.float64)


In [37]:
y_obs = np.array([[[2],[2],[3]]]) #Needa 1 Dim, 3 timesteps, 1 output per timestep
x_for = np.array([[[2],[100],[3]],[[2],[2],[3]],[[2],[2],[3]]]) #2 Models, each with 3 timesteps, and 1 output per timestep

#if one of the models is the same as the path, value doesn't adjust

y_obs = np.cumsum(y_obs, axis=1)
x_for = np.cumsum(x_for, axis=1)

X = torch.tensor(x_for, dtype=torch.double)
y = torch.tensor(y_obs, dtype=torch.double)

print("X:", X)
print("y:", y)

testkernel = sigkernel.LinearKernel()
K_Xy = testkernel.Gram_matrix(X,y)
K_XX = testkernel.Gram_matrix(X,X)
print("X,y", K_Xy)
print("X,X", K_XX)

print(K_XX.shape[0])

K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))


sig_kernel = sigkernel.SigKernel(testkernel, dyadic_order)


X: tensor([[[  2.],
         [102.],
         [105.]],

        [[  2.],
         [  4.],
         [  7.]],

        [[  2.],
         [  4.],
         [  7.]]], dtype=torch.float64)
y: tensor([[[2.],
         [4.],
         [7.]]], dtype=torch.float64)
X,y tensor([[[[  4.,   8.,  14.],
          [204., 408., 714.],
          [210., 420., 735.]]],


        [[[  4.,   8.,  14.],
          [  8.,  16.,  28.],
          [ 14.,  28.,  49.]]],


        [[[  4.,   8.,  14.],
          [  8.,  16.,  28.],
          [ 14.,  28.,  49.]]]], dtype=torch.float64)
X,X tensor([[[[4.0000e+00, 2.0400e+02, 2.1000e+02],
          [2.0400e+02, 1.0404e+04, 1.0710e+04],
          [2.1000e+02, 1.0710e+04, 1.1025e+04]],

         [[4.0000e+00, 8.0000e+00, 1.4000e+01],
          [2.0400e+02, 4.0800e+02, 7.1400e+02],
          [2.1000e+02, 4.2000e+02, 7.3500e+02]],

         [[4.0000e+00, 8.0000e+00, 1.4000e+01],
          [2.0400e+02, 4.0800e+02, 7.1400e+02],
          [2.1000e+02, 4.2000e+02, 7.3500e+02]]]

RuntimeError: diag(): Supports 1D or 2D tensors. Got 4D

In [ ]:
dim_t = int(y_obs[0][0])
dim_total = y_obs[0].shape[0]-1
stat_obs, stat_sim = self._calculate_summary_stat(y_obs, y_sim)

# Reshape and compute cumulative sums of statistics

stat_obs = np.dstack([np.cumsum(i[1:].reshape(int(dim_total/dim_t),int(dim_t)).T,axis=0) for i in stat_obs])
stat_sim = np.dstack([np.cumsum(i[1:].reshape(int(dim_total/dim_t),int(dim_t)).T,axis=0) for i in stat_sim])



stat_obs = rearrange(stat_obs, 't b d -> d t b')
stat_sim = rearrange(stat_sim, 't b d -> d t b')

# # need (obs,time,snp) shape

# # Tuning parameter for RBF kernel
# 
static_kernel = sigkernel.RBFKernel(sigma=self.sigma) # hilbert kernel uplift
dyadic_order = 3


signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

# compute the scoring rule

sr = signature_kernel.compute_scoring_rule(torch.tensor(stat_sim, dtype=torch.float64),torch.tensor(stat_obs, dtype=torch.float64))